In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

# Module 11 Lab - Model Evaluation

## Directions


The due dates for each are indicated in the Syllabus and the course calendar. If anything is unclear, please email EN685.648@gmail.com the official email for the course or ask questions in the Lab discussion area on Blackboard.

The Labs also present technical material that augments the lectures and "book".  You should read through the entire lab at the start of each module.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Please follow the directions and make sure you provide the requested output. Failure to do so may result in a lower grade even if the code is correct or even 0 points.
</div>

1. Show all work/steps/calculations using Code and Markdown cells.
2. Submit your notebook (.ipynb).
3. You may use any core Python libraries or Numpy/Scipy. **Additionally, code from the Module notebooks and lectures is fair to use and modify.** You may also consult Stackoverflow (SO). If you use something from SO, please place a comment with the URL to document the code.

In [3]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import random
import patsy
import sklearn.linear_model as linear

sns.set(style="whitegrid")


# load whatever other libraries you need including models.py
import models

## Model Evaluation and Improvement

As we saw in both the Linear Regression and Logistic Regression modules, there is a Statistician's view of Model Evaluation (and perhaps, Improvement) and a Machine Learning view of Model Evaluation and Improvement.

We'll be working with the **insurance data**.

**1. Load the data, perform your transformations, and using the Bootstrap version of the Linear Regression function, estimate your final model from Lab 10 and show the Bootstrap results**

Let's use the model produced at the end of the Lab 10 solution:

"charges ~ age_sq + male + bmi + smoke_yes + smoke_yes:bmi + smoke_yes:bmi_above_30 + children"

Let's load up the data, and create the transformations listed

In [4]:
data = pd.read_csv("insurance.csv")

data["age_sq"]       = data.age * data.age
data["smoke_yes"]    = pd.get_dummies(data.smoker, prefix="smoke").smoke_yes
data["male"]         = pd.get_dummies(data.sex).male
data["bmi_above_30"] = (data.bmi > 30).map({True: 1, False: 0})
data

,age,sex,bmi,children,smoker,region,charges,age_sq,smoke_yes,male,bmi_above_30
0,19,female,27.900,0,yes,southwest,16884.92400,361,1,0,0
1,18,male,33.770,1,no,southeast,1725.55230,324,0,1,1
2,28,male,33.000,3,no,southeast,4449.46200,784,0,1,1
3,33,male,22.705,0,no,northwest,21984.47061,1089,0,1,0
4,32,male,28.880,0,no,northwest,3866.85520,1024,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830,2500,0,1,1
1334,18,female,31.920,0,no,northeast,2205.98080,324,0,0,1
1335,18,female,36.850,0,no,southeast,1629.83350,324,0,0,1
1336,21,female,25.800,0,no,southwest,2007.94500,441,0,0,0


In [5]:
model = "charges ~ age_sq + male + bmi + smoke_yes + smoke_yes:bmi + smoke_yes:bmi_above_30 + children"
final = models.bootstrap_linear_regression(model, data=data)
models.describe_bootstrap_lr(final)

Our results are indeed pretty good. 

**2. Perform three rounds of 10-fold cross validation, estimating $R^2$ and $\sigma$ each round. Using the results for the test data, calculate 95% Bootstrap estimates of the credible intervals for each.** Comment on these intervals and the intervals from above. Are the average values different? Are the intervals different?

To do this, we'll use sklearn's `cross_val_score`. However, first we need to get our model into a form that sklearn can take in and evaluate. 

In [6]:
y, X = patsy.dmatrices(model, data, return_type="matrix")
reg = linear.LinearRegression()

In [7]:
from sklearn.model_selection import cross_val_score

In [8]:
cross_val_score(estimator = reg, X = X, y = y, cv = 10)

array([0.90857843, 0.87877288, 0.84849802, 0.77002466, 0.87131877,
       0.93295852, 0.88713419, 0.81778739, 0.88005958, 0.86997125])

sklearn's `LinearRegression` uses $R^2$ as its scoring method, so the numbers we see above are the various $R^2$ values for each fold. But we need both $R^2$ and $\sigma$, so let's define our own scoring function that calculates $\sigma$ and combine those scores with our $R^2$ scores.

In [9]:
def calc_sigma(model, X, y):
    result = {}
    result["coefficients"] = model.coef_[0]

    result["r_squared"] = model.score(X, y)
    y_hat = model.predict(X)
    result["residuals"] = y - y_hat
    
    sum_squared_error = sum(e ** 2 for e in result["residuals"])[0]

    n = len(result["residuals"])
    k = len(result["coefficients"])

    sigma = np.sqrt(sum_squared_error / (n - k))
    return sigma

In [10]:
cross_val_score(estimator = reg, X = X, y = y, cv = 10, scoring = calc_sigma)

array([4028.27320942, 4300.94275776, 4745.9210399 , 5390.33820938,
       4810.78785585, 2963.6001118 , 4224.55859   , 5325.7129349 ,
       4063.46071938, 4926.55737718])

Let's put that together in a single function that estimates $R^2$ and $\sigma$. To estimate

In [11]:
def CV_estimate_r2_sigma(estimator, X, y, n_folds):
    pass

In [12]:
def bootstrap_sample(data, f, n=100):
    m = len(data)
    return np.array(
        [f(np.random.choice(data, len(data), replace=True)) for _ in range(n)]
    )

**3. Using Learning Curves and $\sigma$ determine if more data will improve the estimation of the model.**

**4. It was shown that `age_sq` improved the performance of the model. Perhaps a different polynomial would have been better. Generate Validation Curves for `age` = [1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5] and select the best transformation.**

**5. Using Ridge Regression to estimate a model for the insurance data. Compare it with your final Linear Regression model.** (If you get far ahead, you may need to write your own function. Here are the sklearn docs: http://scikit-learn.org/stable/modules/linear_model.html)